# Purpose

# Imports

In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '../automatic-pancake/data-jupyter-notebook@athena-179008.json'

In [2]:
import pandas as pd
import numpy as np
import ast
from datetime import datetime as dt
import json
from google.cloud import bigquery
import time
import os,sys
import warnings
import requests

warnings.filterwarnings('ignore')
autostats_lib = "../autostats/"

if autostats_lib not in sys.path:
    sys.path.append(autostats_lib)
    
helpers_lib = "../automatic-pancake/helpers/"

if helpers_lib not in sys.path:
    sys.path.append(helpers_lib)
    
ruleengine_helpers_lib = "../rule-engine/testing/"

if ruleengine_helpers_lib not in sys.path:
    sys.path.append(ruleengine_helpers_lib)

from feature_scores import FeatureScores
from testing.test_helpers import get_distribution,get_numeric_feature_slug_name,get_token,get_namespace_result,get_distribution_batch
def display_full(x):
    pd.set_option('display.max_rows', len(x))
    display(x)
    pd.reset_option('display.max_rows')

import qgrid

%reload_ext google.cloud.bigquery

## reload helper

In [3]:
import testing.test_helpers as helpers
import importlib
importlib.reload(helpers)

<module 'testing.test_helpers' from '/mnt/disks/disk1/jupyter-yohanes.gosal@indo-11865_base/indodana-data-analysis/rule-engine/testing/test_helpers.py'>

In [143]:
import gc
gc.collect()

20

In [4]:
autostats_lib = "../autostats/"
if autostats_lib not in sys.path:
    sys.path.append(autostats_lib)
    
from utilities.data_storage import DataStorage
from itertools import chain,cycle
from IPython.core.display import display, HTML, display_html
from scorecard.scorecard import Scorecard
from scorecard.scoring_model import ScoringModel
from scorecard.binning_builder import BinningBuilder

In [5]:
def display_full(x, columns=False, amount_columns=[], percentage_columns=[], ):
    pd.set_option('display.max_rows', len(x))
    if columns:
        pd.set_option('display.max_columns', len(x.columns))
    
    try:
        all_cols = percentage_columns + amount_columns
        
        if percentage_columns == []:
            all_cols = x.columns.to_list()

            percentage_cols = [
                'PERC',
                'APPROVAL_RATE',
                'UTILIZATION_RATE',
                'badrate15amt'
            ]
            badrate_cols = x.columns[x.columns.str.contains('BADRATE')].to_list()
            percentage_cols = percentage_cols + badrate_cols
        else:
            percentage_cols = percentage_columns
            
        amount_cols = amount_columns + ['total_amt'] + ['limit_exposure'] + ['limit_exposure_old']

        for col in percentage_cols:
            if col in all_cols:
                if isinstance(x[col], pd.Series):
                    x[col] = x[col].map('{:,.2%}'.format)
                elif isinstance(x[col], pd.DataFrame):
                    x[col] = x[col].applymap('{:,.2%}'.format)

        for col in amount_cols:
            if col in all_cols:
                x[col] = x[col].map('{:,.0f}'.format)
#                 x[col] = x[col].apply(short_currency_format)
        display(x.style.set_sticky(axis="columns"))
        
    except:      
        display(x.style.set_sticky(axis="columns"))

    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')

## display setting

In [6]:
from IPython.display import display, HTML

display(HTML("<style>div.output_scroll { height: 55em; }</style>")) 
display(HTML("<style>.container { width:85% !important; }</style>"))

## Get staging token

In [7]:
tok = get_token(is_staging=False, login_file='login.txt')

# Functions

In [8]:
def load_dataframe_to_testpandasgbq(data):
    client = bigquery.Client(project='athena-179008')
    table_id = 'vayu_data_mart.testpandasgbq'
    job_config = bigquery.LoadJobConfig(schema=[
    #     bigquery.SchemaField("finalScore", "STRING"),
    ],
    write_disposition="WRITE_TRUNCATE",)

    start = time.time()
    # data = pd.DataFrame(order_ids).rename(columns={0 : 'orderId'})

    job = client.load_table_from_dataframe(
        data, table_id
        , job_config=job_config
    )
    # Wait for the load job to complete.
    job.result()
    end = time.time()
    print("upload dataframe takes {}".format(end-start))
    return print('Dataframe Successfully Loaded to vayu_data_mart.testpandasgbq')

def print_summary(data, metrics_to_compare):
    for index, row in data.iterrows():
        print(index)
        for m in metrics_to_compare:
#             print(m, row)
#             print(row[m+'_old'])
#             print(row[m+'_new'])
#             print(row[m+'_perc_chg'])

            print(m, ': ', str(row[m+'_old']), ' --> ', str(row[m+'_new']), ' (' + ('+' if '-' not in str(row[m+'_perc_chg']) else '') + str(row[m+'_perc_chg']) + ')')
        print('\n')
        
def load_dataframe_to_data_mart(data, table_name):
    client = bigquery.Client(project='athena-179008')
    table_id = 'athena-179008.vayu_data_mart.' + table_name
    job_config = bigquery.LoadJobConfig(schema=[
    ],
    write_disposition="WRITE_TRUNCATE",)
    start = time.time()
    # data = pd.DataFrame(order_ids).rename(columns={0 : 'orderId'})
    job = client.load_table_from_dataframe(
        data, table_id
        , job_config=job_config
    )
    # Wait for the load job to complete.
    job.result()
    end = time.time()
    print("upload dataframe takes {}".format(end-start))
    return print('Dataframe Successfully Loaded to '+table_name)

## Pivot Functions

### Bad Rate

In [9]:
def bad_rate_pivot(data, data_type, flags=None, index=None):
    if data_type in ['application cli', 'application cash', 'application trx']:
        pivots = []
        
        aggfunc = 'count'
        if data_type == 'application trx':
            aggfunc = {'orderId': 'nunique'}
            
        for flag in flags:
            pivot = pd.pivot_table(data,
                                   index=index,
                                   columns=flag,
                                   values='orderId',
                                   aggfunc=aggfunc,
#                                    fill_value=0,
                                   observed=True
                                   )
            split_flag = flag.split("_")
            pivot['matured_' + split_flag[1] + "_" + split_flag[2]] = pivot[1] + pivot[0]
            pivot['pd_' + split_flag[1] + "_" + split_flag[2]] = pivot[1] / (pivot[1] + pivot[0])
            dpd_cols = pivot.columns[pivot.columns.astype(str).str.contains('pd|matured')]
            pivots.append(pivot[dpd_cols])
        
        pivot = pd.concat(pivots, axis=1)
        return pivot
    
    if data_type == 'transaction':
        pivots = []
        for flag in flags:
            pivot = pd.pivot_table(data,
                                   index=index,
                                   columns=flag,
                                   values='entity_id',
                                   aggfunc='count',
                                   fill_value=0,
                                   observed=True
                                   )
            split_flag = flag.split("_")
            pivot['matured_' + split_flag[1] + "_" + split_flag[2]] = pivot[1] + pivot[0]
            pivot['pd_' + split_flag[1] + "_" + split_flag[2]] = pivot[1] / (pivot[1] + pivot[0])
            dpd_cols = pivot.columns[pivot.columns.astype(str).str.contains('pd|matured')]
            pivots.append(pivot[dpd_cols])
        
        pivot = pd.concat(pivots, axis=1)
        return pivot
    
def bad_rate_amt_pivot(data, data_type, flags=None, index=None, amount_col=None):
    if data_type == 'application cli':
        badrate_amt_pivot = pd.pivot_table(data,
                                           index=index,
                                           values=[
                                               'sum_loan_amount_bad_15_installment_1',
                                               'sum_loan_amount_good_15_installment_1'
                                           ],
                                           aggfunc='sum',
                                           fill_value=0,
                                           observed=True
                                        )

        badrate_amt_pivot['matured_15_1_amt'] = badrate_amt_pivot['sum_loan_amount_bad_15_installment_1'] + badrate_amt_pivot['sum_loan_amount_good_15_installment_1']
        badrate_amt_pivot['pd_15_1_amt'] = badrate_amt_pivot['sum_loan_amount_bad_15_installment_1'] / badrate_amt_pivot['matured_15_1_amt'].replace({0: np.nan})

        dpd_cols = badrate_amt_pivot.columns[badrate_amt_pivot.columns.astype(str).str.contains('pd|matured')]
        return badrate_amt_pivot[dpd_cols]
    
    if data_type == 'application cash':
        if amount_col == None:
            amount_col = 'approvedloanamount'
        
        pivots = []
        
        for flag in flags:
            pivot = pd.pivot_table(data,
                                   index=index,
                                   columns=flag,
                                   values=amount_col,
                                   aggfunc='sum',
                                   fill_value=0,
                                   observed=True
                                   )
            split_flag = flag.split("_")
            pivot['matured_' + split_flag[1] + "_" + split_flag[2] + "_amt"] = pivot[1] + pivot[0]
            pivot['pd_' + split_flag[1] + "_" + split_flag[2] + "_amt"] = pivot[1] / (pivot[1] + pivot[0])
            dpd_cols = pivot.columns[pivot.columns.astype(str).str.contains('pd|matured')]
            pivots.append(pivot[dpd_cols])
            
        pivot = pd.concat(pivots, axis=1)
        return pivot
    
    if data_type in ['transaction', 'application trx']:
        
        pivots = []
        
        for flag in flags:
            pivot = pd.pivot_table(data,
                                   index=index,
                                   columns=flag,
                                   values='loan_amount',
                                   aggfunc='sum',
                                   fill_value=0,
                                   observed=True
                                   )
            split_flag = flag.split("_")
            pivot['matured_' + split_flag[1] + "_" + split_flag[2] + "_amt"] = pivot[1] + pivot[0]
            pivot['pd_' + split_flag[1] + "_" + split_flag[2] + "_amt"] = pivot[1] / (pivot[1] + pivot[0])
            dpd_cols = pivot.columns[pivot.columns.astype(str).str.contains('pd|matured')]
            pivots.append(pivot[dpd_cols])
            
        pivot = pd.concat(pivots, axis=1)
        return pivot
    
def loss_pivot(data, index):
    loss_pivot = pd.pivot_table(data,
                                index=index,
                                values=[
                                    'loan_amount_due',
                                    'total_payment_amount'
                                ],
                                aggfunc='sum',
                                fill_value=0,
                                observed=True)

    loss_pivot['principal_loss'] = 1 - (loss_pivot['total_payment_amount'] / loss_pivot['loan_amount_due'])
    return loss_pivot

### Complete

In [10]:
def complete_pivot(data, data_type, data_filters=None, index='default', flags='default', default_month_column='submitdate_month', loan_amount_col=None, limit_col=None, badrate=False, badrate_amount=False, loss=False, approval_rate=False, cancelled_rate=False, utilization_rate=False, groupby_levels=[0, 1], custom_param={}):
    
    if isinstance(data_filters, pd.Series):
            data = data[data_filters]
    
    if flags == 'default':
        flags = [
            'flag_15_1'
        ]
        
    if index == 'default':
        if default_month_column == 'submitdate_month':
            index = [
                'business_segment',
                'submitdate_month',
            ]
        else:
            index = [
                'business_segment',
                default_month_column
            ]
    
    values = ['orderId']
    aggfunc = {'orderId': 'count'}
    
    if 'summary' in custom_param.keys():
        if 'values' in custom_param['summary']:
            values = custom_param['summary']['values']
        if 'aggfunc' in custom_param['summary']:
            aggfunc = custom_param['summary']['aggfunc']
        
    if loan_amount_col != None:
        values.append(loan_amount_col)
        aggfunc[loan_amount_col] = 'sum' 
    
    if limit_col != None:
        values.append(limit_col)
        aggfunc[limit_col] = 'mean' 
        
    summary_pivot = pd.pivot_table(data,
                                   index=index,
                                   values=values,
                                   aggfunc=aggfunc,
                                   fill_value=0,
                                   observed=True
                                  )
    summary_pivot['orderId_cnt'] = summary_pivot['orderId']
    summary_pivot['orderId_cnt_dist'] = summary_pivot['orderId'] / summary_pivot['orderId'].groupby(level=groupby_levels).transform('sum')
    summary_pivot['orderId_cnt_cumdist'] = summary_pivot['orderId_cnt_dist'].groupby(level=groupby_levels).transform('cumsum')
    
    if loan_amount_col != None:
        summary_pivot['total_loan_amt'] = summary_pivot[loan_amount_col]
        summary_pivot['loan_amt_dist'] = summary_pivot[loan_amount_col] / summary_pivot[loan_amount_col].groupby(level=groupby_levels).transform('sum')
    
    if limit_col != None:
        summary_pivot['avg_' + limit_col] = summary_pivot[limit_col]
    
    if badrate == True:
        badrate_pivot = bad_rate_pivot(data, data_type, flags=flags, index=index)

    if badrate_amount == True:
        badrate_amt_pivot = bad_rate_amt_pivot(data, data_type, flags=flags, index=index, amount_col=loan_amount_col)
    
    if loss == True:
        amt_loss_pivot = loss_pivot(data, index)

    
    if approval_rate == True:
        values = 'orderId'
        aggfunc = {'orderId': 'count'}
        if 'summary_approval' in custom_param.keys():
            if 'values' in custom_param['summary_approval']:
                values = custom_param['summary_approval']['values']
            if 'aggfunc' in custom_param['summary_approval']:
                aggfunc = custom_param['summary_approval']['aggfunc']
                
        summary_pivot_2 = pd.pivot_table(data,
                                         index=index,
                                         columns='is_approved',
                                         values=values,
                                         aggfunc=aggfunc,
                                         fill_value=0,
                                         observed=True
                                        )
        summary_pivot_2['orderId_approved_cnt'] = summary_pivot_2[1]
        summary_pivot_2['orderId_approved_cnt_dist'] = summary_pivot_2['orderId_approved_cnt'] / summary_pivot_2['orderId_approved_cnt'].groupby(level=groupby_levels).transform('sum')
        summary_pivot_2['orderId_approved_cnt_cumdist'] = summary_pivot_2['orderId_approved_cnt_dist'].groupby(level=groupby_levels).transform('cumsum')
        if 0 in summary_pivot_2.columns:
            summary_pivot_2['approval_rate'] = summary_pivot_2[1] / (summary_pivot_2[1] + summary_pivot_2[0])
            summary_pivot_2.drop([0, 1], axis=1, inplace=True)
        else:
            summary_pivot_2['approval_rate'] = summary_pivot_2[1] / (summary_pivot_2[1])
            summary_pivot_2.drop([1], axis=1, inplace=True)
            
    
    if cancelled_rate == True:
        summary_pivot_3 = pd.pivot_table(data,
                                         index=index,
                                         columns='is_cancelled',
                                         values='orderId',
                                         aggfunc=({'orderId': 'count'}),
                                         fill_value=0,
                                         observed=True
                                        )
        summary_pivot_3['orderId_cancelled_cnt'] = summary_pivot_3[1]
        summary_pivot_3['orderId_cancelled_cnt_dist'] = summary_pivot_3['orderId_cancelled_cnt'] / summary_pivot_3['orderId_cancelled_cnt'].groupby(level=groupby_levels).transform('sum')
        summary_pivot_3['orderId_cancelled_cnt_cumdist'] = summary_pivot_3['orderId_cancelled_cnt_dist'].groupby(level=groupby_levels).transform('cumsum')
        summary_pivot_3.drop([0, 1], axis=1, inplace=True)
        
    if utilization_rate == True:
        summary_pivot_4 = pd.pivot_table(data,
                                         index=index,
                                         columns='is_used',
                                         values='orderId',
                                         aggfunc=({'orderId': 'count'}),
                                         fill_value=0,
                                         observed=True
                                        )
        if 1 in summary_pivot_2.columns:
            summary_pivot_4['orderId_utilized_cnt'] = summary_pivot_4[1]
            summary_pivot_4['orderId_utilized_cnt_dist'] = summary_pivot_4['orderId_utilized_cnt'] / summary_pivot_4['orderId_utilized_cnt'].groupby(level=groupby_levels).transform('sum')
            summary_pivot_4['orderId_utilized_cnt_cumdist'] = summary_pivot_4['orderId_utilized_cnt_dist'].groupby(level=groupby_levels).transform('cumsum')
        else:
            summary_pivot_4['orderId_utilized_cnt'] = 0
        
        for x in [0, 1]:
            if x in summary_pivot_4.columns:
                summary_pivot_4.drop([x], axis=1, inplace=True)
        
    
    final_pivot_list = [summary_pivot]
    
    if badrate == True: 
        final_pivot_list.append(badrate_pivot)
        
    if badrate_amount == True:
        final_pivot_list.append(badrate_amt_pivot)
        
    if loss == True:
        final_pivot_list.append(amt_loss_pivot)
        
    if approval_rate == True:
        final_pivot_list.append(summary_pivot_2)

    if cancelled_rate == True:
        final_pivot_list.append(summary_pivot_3)

    if utilization_rate == True:
        final_pivot_list.append(summary_pivot_4)

    final_pivot = pd.concat(final_pivot_list, axis=1)

    if (cancelled_rate == True) and (approval_rate == True):
        final_pivot['cancelled_rate'] = final_pivot['orderId_cancelled_cnt'] / final_pivot['orderId_approved_cnt']
    
    if (utilization_rate == True) and (approval_rate == True):
        final_pivot['utilized_rate'] = final_pivot['orderId_utilized_cnt'] / final_pivot['orderId_approved_cnt']
        
    if (loan_amount_col != None) and (approval_rate == True):
        final_pivot['avg_loan_amt'] = final_pivot['total_loan_amt'] / final_pivot['orderId_approved_cnt']
        
    return final_pivot

def complete_pivot_trx(data, data_filters=None, index='default', flags='default', default_month_column='trx_approved_date_month', badrate=True, loss=False, groupby_levels=[0, 1]):
    
    if isinstance(data_filters, pd.Series):
            data = data[data_filters]
    
    if flags == 'default':
        flags = [
            'flag_15_1'
        ]
        
    if index == 'default':
        if default_month_column == 'trx_approved_date_month':
            index = [
                'business_segment',
                'trx_approved_date_month',
            ]
        elif default_month_column == 'submitdate_month':
            index = [
                'business_segment',
                'submitdate_month',
            ]
    
    if badrate:
        pivots = []
        for flag in flags:
            pivot = pd.pivot_table(data,
                                   index=index,
                                   columns=flag,
                                   values='entity_id',
                                   aggfunc='count',
                                   fill_value=0,
                                   observed=True
                                   )
            split_flag = flag.split("_")
            pivot['matured_' + split_flag[1] + "_" + split_flag[2]] = pivot[1] + pivot[0]
            pivot['pd_' + split_flag[1] + "_" + split_flag[2]] = pivot[1] / (pivot[1] + pivot[0])
            dpd_cols = pivot.columns[pivot.columns.astype(str).str.contains('pd|matured')]
            pivots.append(pivot[dpd_cols])

            pivot = pd.pivot_table(data,
                                   index=index,
                                   columns=flag,
                                   values='loan_amount',
                                   aggfunc='sum',
                                   fill_value=0,
                                   observed=True
                                   )
            split_flag = flag.split("_")
            pivot['matured_' + split_flag[1] + "_" + split_flag[2] + "_amt"] = pivot[1] + pivot[0]
            pivot['pd_' + split_flag[1] + "_" + split_flag[2] + "_amt"] = pivot[1] / (pivot[1] + pivot[0])
            dpd_cols = pivot.columns[pivot.columns.astype(str).str.contains('pd|matured')]
            pivots.append(pivot[dpd_cols])

        badrate_pivot = pd.concat(pivots, axis=1)
        badrate_pivot.columns.name = 'badrate'
    
    if loss:
        loss_pivot = pd.pivot_table(data,
                                    index=index,
                                    values=[
                                        'loan_amount_due',
                                        'total_payment_amount'
                                    ],
                                    aggfunc='sum',
                                    fill_value=0,
                                    observed=True)
        
        loss_pivot['principal_loss'] = 1 - (loss_pivot['total_payment_amount'] / loss_pivot['loan_amount_due'])
        
        
    summary_pivot = pd.pivot_table(data, 
                      index=index,
                      values=['entity_id', 'orderId', 'userId', 'loan_amount'], 
                      aggfunc={'entity_id': 'count', 'orderId': 'nunique', 'userId': 'nunique', 'loan_amount': 'sum'},
                      observed=True
                     )
    summary_pivot['trx_cnt'] = summary_pivot['entity_id']
    summary_pivot['trx_cnt_dist'] = summary_pivot['entity_id'] / summary_pivot['entity_id'].groupby(level=groupby_levels).transform('sum')
    summary_pivot['trx_cnt_cumdist'] =summary_pivot['trx_cnt_dist'].groupby(level=groupby_levels).transform('cumsum')
    summary_pivot['orderId_cnt'] = summary_pivot['orderId']
    summary_pivot['orderId_cnt_dist'] = summary_pivot['orderId'] / summary_pivot['orderId'].groupby(level=groupby_levels).transform('sum')
    summary_pivot['orderId_cnt_cumdist'] =summary_pivot['orderId_cnt_dist'].groupby(level=groupby_levels).transform('cumsum')
    summary_pivot['userId_cnt'] = summary_pivot['userId']
    summary_pivot['userId_cnt_dist'] = summary_pivot['userId'] / summary_pivot['userId'].groupby(level=groupby_levels).transform('sum')
    summary_pivot['userId_cnt_cumdist'] =summary_pivot['userId_cnt_dist'].groupby(level=groupby_levels).transform('cumsum')
    summary_pivot['total_trx_amt'] = summary_pivot['loan_amount']
    summary_pivot['trx_amt_dist'] = summary_pivot['loan_amount'] / summary_pivot['loan_amount'].groupby(level=groupby_levels).transform('sum')
    summary_pivot['avg_trx_amt'] = summary_pivot['loan_amount'] / summary_pivot['entity_id']
    
    pivot_list = []
    if badrate:
        pivot_list.append(badrate_pivot)
    if loss:
        pivot_list.append(loss_pivot)
    
    pivot_list.append(summary_pivot)
    final_pivot = pd.concat(pivot_list, axis=1)
    
    return final_pivot

### Formatting

In [11]:
def format_pivot(pivot, pivot_filters=None, sort_by=[], ascending=[], amount_columns='default', percentage_columns='default', columns_to_hide=[], columns_to_display=[], custom_format={}):
    
    if isinstance(pivot_filters, pd.Series):
        pivot = pivot[pivot_filters]

    if amount_columns == 'default':
        amount_columns = pivot.columns[
            ((pivot.columns.astype(str).str.contains('amt'))
             | (pivot.columns.astype(str).str.contains('amount'))
            )
            & (~pivot.columns.astype(str).str.contains('pd'))
            & (~pivot.columns.astype(str).str.contains('dist'))
            & (~pivot.columns.astype(str).str.contains('pct_chg'))
        ].to_list()
        
    if percentage_columns == 'default':
        percentage_columns = pivot.columns[
            (pivot.columns.astype(str).str.contains('pd'))
            | (pivot.columns.astype(str).str.contains('dist'))
            | (pivot.columns.astype(str).str.contains('rate'))
            | (pivot.columns.astype(str).str.contains('loss'))
            | (pivot.columns.astype(str).str.contains('pct_chg'))
            | (pivot.columns.astype(str).str.contains('perc'))
        ].to_list()
    
    if len(sort_by) > 0:
        if ascending != []:
            pivot = pivot.sort_values(by=sort_by, ascending=ascending)
        else:
            pivot = pivot.sort_values(by=sort_by, ascending=True)
            
    pivot[amount_columns] = pivot[amount_columns].applymap('{:,.2f}'.format)
    pivot[percentage_columns] = pivot[percentage_columns].applymap('{:,.2%}'.format)
    
    if custom_format != {}:
        for col, f in zip(custom_format.keys(), custom_format.values()):
            pivot[col] = pivot[col].apply(f.format)
    
    if len(columns_to_display) < 1:
        columns_to_display = pivot.columns[~pivot.columns.isin(columns_to_hide)].to_list()
    else:
        for col in columns_to_hide:
            if col in columns_to_display:
                columns_to_display.remove(col)
    return pivot[columns_to_display]

# Data Import

In [12]:
data = pd.read_csv('final_df_va_rbp_2025_10_13.csv')
data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91024 entries, 0 to 91023
Data columns (total 46 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   result_old                              91024 non-null  object 
 1   identifier                              91024 non-null  object 
 2   resultType_old                          91024 non-null  object 
 3   result_new                              45779 non-null  object 
 4   resultType_new                          45779 non-null  object 
 5   id                                      91024 non-null  object 
 6   function_id                             91024 non-null  object 
 7   namespace                               91024 non-null  object 
 8   result                                  91024 non-null  object 
 9   reference                               91024 non-null  object 
 10  version                                 91024 non-null  in

In [37]:
results_rbp_old_df = pd.read_csv('results_user_rbp_v67_trx_2025_10_06.csv')

results_rbp_new_df = pd.read_csv('results_user_rbp_v73_trx_2025_10_06.csv')

In [55]:
final_df_analysis = pd.read_csv('final_df_va_rbp_2025_10_13.csv')

In [56]:
final_df_analysis.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91024 entries, 0 to 91023
Data columns (total 52 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   result_old                              91024 non-null  object 
 1   identifier                              91024 non-null  object 
 2   resultType_old                          91024 non-null  object 
 3   result_new                              45779 non-null  object 
 4   resultType_new                          45779 non-null  object 
 5   result_old_ori_data                     91024 non-null  object 
 6   identifier_ori_data                     91024 non-null  object 
 7   resultType_old_ori_data                 91024 non-null  object 
 8   result_new_ori_data                     45779 non-null  object 
 9   resultType_new_ori_data                 45779 non-null  object 
 10  id                                      91024 non-null  ob

## Data Preparation

In [28]:
data.groupby(['merchant_name']).agg({'flogs_created_at':['min', 'max'], 'reference':['count', 'nunique']})

flogs_created_at  \
                                                                               min   
merchant_name                                                                        
101 02 Panakukang Mall                            2025-10-03 07:53:28.775757+00:00   
101 03 Karawaci Supermall                         2025-10-04 04:24:47.307714+00:00   
4 EVER FIT 01 - SUMMARECON BEKASI                 2025-10-02 09:34:58.558856+00:00   
6IXTY8IGHT 04 - TRANS STUDIO MALL MAKASSAR        2025-10-02 09:33:18.341072+00:00   
8 J STORE 01 - DENPASAR                           2025-10-02 05:08:19.495711+00:00   
...                                                                            ...   
ZOA INTERSTELLAR GROUP 001 - UP GADGET PONTIANAK  2025-10-02 03:39:22.976746+00:00   
ZONA IPHONE 001-TASIKMALAYA                       2025-10-05 14:05:18.567351+00:00   
Zglow Clinic 09 Cipinang                          2025-10-02 07:25:35.142312+00:00   
pasarwarga                                        2025-10-05 08:42:46.746155+00:00   
ruparupa                                          2025-10-02 04:27:15.770282+00:00   

                                                                                    \
                                                                               max   
merchant_name                                                                        
101 02 Panakukang Mall                            2025-10-05 07:55:42.491779+00:00   
101 03 Karawaci Supermall                         2025-10-04 04:24:47.307714+00:00   
4 EVER FIT 01 - SUMMARECON BEKASI                 2025-10-05 07:48:15.087288+00:00   
6IXTY8IGHT 04 - TRANS STUDIO MALL MAKASSAR        2025-10-02 09:33:18.341072+00:00   
8 J STORE 01 - DENPASAR                           2025-10-05 14:05:01.227783+00:00   
...                                                                            ...   
ZOA INTERSTELLAR GROUP 001 - UP GADGET PONTIANAK  2025-10-05 14:38:28.767913+00:00   
ZONA IPHONE 001-TASIKMALAYA                       2025-10-05 14:05:18.567351+00:00   
Zglow Clinic 09 Cipinang                          2025-10-02 07:29:38.170185+00:00   
pasarwarga                                        2025-10-05 08:42:46.746155+00:00   
ruparupa                                          2025-10-05 22:22:22.641010+00:00   

                                                 reference          
                                                     count nunique  
merchant_name                                                       
101 02 Panakukang Mall                                   4       4  
101 03 Karawaci Supermall                                1       1  
4 EVER FIT 01 - SUMMARECON BEKASI                       13      13  
6IXTY8IGHT 04 - TRANS STUDIO MALL MAKASSAR               1       1  
8 J STORE 01 - DENPASAR                                 23      23  
...                                                    ...     ...  
ZOA INTERSTELLAR GROUP 001 - UP GADGET PONTIANAK        53      53  
ZONA IPHONE 001-TASIKMALAYA                              1       1  
Zglow Clinic 09 Cipinang                                 3       3  
pasarwarga                                               1       1  
ruparupa                                               108     108  

[4440 rows x 4 columns]

In [61]:
data[data['userLastActivitySegment'] != '-9999'][[
    'reference',
    'information'
]].to_csv('user_rbp_2025_10_06.csv')

## Current Rule

### userRateGroupKeyByGradingRules


In [ ]:
result_rbp = helpers.get_distribution_batch('user_rbp_2025_10_06.csv', 
                                           tok,
                                            dataset='transaction',
                                            orderid='reference',
                                           version=67,
                                           name='userRateGroupKeyByGradingRules',
                                            namespace='indodana.rules.general.implementation',
                                           is_staging=False,
#                                                chunk_size=3000,
                                           verbose=False,
#                                            v4=True,
                                           v5=True,
                                           max_file_size=45
                                          )

Checking function correctness... [OK]
91139 3.9062585830688477
	>>> maximum batch size: 16500

GOOGLE_APPLICATION_CREDENTIALS: ../automatic-pancake/data-jupyter-notebook@athena-179008.json
GCS_BUCKET: cermati-indodana-risk-data-analytic
Connection initialized to Google Cloud Storage project athena-179008 and bucket cermati-indodana-risk-data-analytic

List of inputs:
1759723355_232a667fe596444db820c2c18726a0b0_0.json: 217.93548393249512 MB
Some files are over 50 MB, reduce chungsize: 16000. Retry...

List of inputs:
1759723379_8bd561d66230437ba28cb442dc95fb55_0.json: 204.73627758026123 MB
Some files are over 50 MB, reduce chungsize: 15500. Retry...

List of inputs:
1759723403_395592a510824764a8c835cd3d63527e_0.json: 192.08111190795898 MB
Some files are over 50 MB, reduce chungsize: 15000. Retry...

List of inputs:
1759723446_d659cb911944430ba1335370b0edf515_0.json: 180.29020404815674 MB
Some files are over 50 MB, reduce chungsize: 14500. Retry...

List of inputs:
1759723549_bd00861be73

1759724184_f0c417c22b2440d7a4993e9b9c13693e_40.json: 39.87181282043457 MB
1759724184_f0c417c22b2440d7a4993e9b9c13693e_41.json: 39.18440341949463 MB
1759724184_f0c417c22b2440d7a4993e9b9c13693e_42.json: 23.177892684936523 MB
1759724184_f0c417c22b2440d7a4993e9b9c13693e_43.json: 1.9221582412719727 MB
1759724184_f0c417c22b2440d7a4993e9b9c13693e_44.json: 1.9297361373901367 MB
1759724184_f0c417c22b2440d7a4993e9b9c13693e_45.json: 23.55862331390381 MB
1759724184_f0c417c22b2440d7a4993e9b9c13693e_46.json: 40.37067890167236 MB
1759724184_f0c417c22b2440d7a4993e9b9c13693e_47.json: 38.96198844909668 MB
1759724184_f0c417c22b2440d7a4993e9b9c13693e_48.json: 23.782614707946777 MB
1759724184_f0c417c22b2440d7a4993e9b9c13693e_49.json: 1.9204273223876953 MB
1759724184_f0c417c22b2440d7a4993e9b9c13693e_5.json: 35.24427318572998 MB
1759724184_f0c417c22b2440d7a4993e9b9c13693e_50.json: 1.9195575714111328 MB
1759724184_f0c417c22b2440d7a4993e9b9c13693e_51.json: 19.482335090637207 MB
1759724184_f0c417c22b2440d7a4993

  0%|          | 0/61 [00:00<?, ?it/s]

In [39]:
ids_rbp = result_rbp[0]
ids_rbp

['bf7e5cb9-974f-4935-8f40-90612bdf2296',
 '9b2e3c4e-ce3f-49c4-a76d-90508772ccc0',
 '10b18db9-fd50-4f15-94bc-5de10b571a45',
 '0e81c095-e9b6-4eba-8540-163a830727fe',
 'f63b2db1-ce35-430c-988f-59a75f4c8e73',
 '1d0adcc8-6c34-4d1d-8f0b-87680dacdb81',
 'c7c0ef96-e567-41ca-8627-5129fcc5a842',
 '5443ad43-89ad-412c-b64f-77fd6f02c74f',
 '223ad3c8-6dff-4602-9968-e4c82ab80244',
 '6a9954bc-30b5-481f-bea3-6de4262a828f',
 '8f4fe0ab-f9cc-4c68-ba9a-18ed0c5d5427',
 'ce38c4ad-eedb-4484-809c-9c9c93d6c9e9',
 'e1d4e8c9-77d6-4d55-bde6-0367aab0b9ef',
 '60ac5a1c-1bc6-49ea-81bc-e310ff8a4a41',
 '1508dad5-7f80-4841-a90f-7a72ae99e406',
 'b61a22cc-eb03-4fba-8a57-6345174a57b7',
 'd8a03851-ee0c-4efc-8c3f-7e5f7edd5522',
 'e558d465-bd6c-4cfb-b39a-e2672a39e538',
 '3ad39e08-93c9-48d4-bdd6-d05b7d67097d',
 '490bd57b-29a0-4936-b6ea-687042096d4e',
 '92d86032-1d91-4aaf-8fe3-521e379c733a',
 'ac10fef6-e503-4ad6-a8dc-7d5beaef613c',
 '922b905b-4d77-4811-af05-0938e076f2ba',
 'af9e0466-c252-4d0a-a7bf-07f6fec0617c',
 '52298819-93d7-

In [44]:
success_rbp, output_rbp, input_rbp = helpers.get_evaluation_results_gcs(ids_rbp, 
                                                                        tok,
                                                                        path='user_rbp_2025_10_06.csv',
                                                                        v5=True
                                                                       )

GOOGLE_APPLICATION_CREDENTIALS: ../automatic-pancake/data-jupyter-notebook@athena-179008.json
GCS_BUCKET: cermati-indodana-risk-data-analytic
Connection initialized to Google Cloud Storage project athena-179008 and bucket cermati-indodana-risk-data-analytic


1. bf7e5cb9-974f-4935-8f40-90612bdf2296
	>>> Evaluation status: DONE
time:2025-10-06 06:45:55.426738
2. 9b2e3c4e-ce3f-49c4-a76d-90508772ccc0
	>>> Evaluation status: DONE
time:2025-10-06 06:45:57.660554
3. 10b18db9-fd50-4f15-94bc-5de10b571a45
	>>> Evaluation status: DONE
time:2025-10-06 06:45:59.663991
4. 0e81c095-e9b6-4eba-8540-163a830727fe
	>>> Evaluation status: DONE
time:2025-10-06 06:46:02.335354
5. f63b2db1-ce35-430c-988f-59a75f4c8e73
	>>> Evaluation status: DONE
time:2025-10-06 06:46:05.754347
6. 1d0adcc8-6c34-4d1d-8f0b-87680dacdb81
	>>> Evaluation status: DONE
time:2025-10-06 06:46:07.910460
7. c7c0ef96-e567-41ca-8627-5129fcc5a842
	>>> Evaluation status: DONE
time:2025-10-06 06:46:09.964658
8. 5443ad43-89ad-412c-b64f-77fd6f

In [45]:
results_df_rbp = pd.DataFrame(output_rbp)
results_df_rbp

,result,identifier,resultType
0,{'offline_merchant_risk_category_group': 'high...,M2BMZ7Q5,OBJECT
1,{'offline_merchant_risk_category_group': 'high...,EWL-KF9AAIAB,OBJECT
2,{'offline_merchant_risk_category_group': 'high...,EWL-SFZHUTYN,OBJECT
3,{'offline_merchant_risk_category_group': 'high...,IDN251005120615271006,OBJECT
4,{'offline_merchant_risk_category_group': 'high...,ID250925-69022627_251002134007229,OBJECT
...,...,...,...
91148,"{'offline_merchant_risk_category_group': 'ok',...",7WJWLZYR,OBJECT
91149,"{'offline_merchant_risk_category_group': 'ok',...",JXNYWSNX,OBJECT
91150,"{'offline_merchant_risk_category_group': 'ok',...",KNWLC93G,OBJECT
91151,"{'offline_merchant_risk_category_group': 'ok',...",XEWUKWQ8,OBJECT


In [46]:
results_df_rbp.to_csv('results_user_rbp_v67_trx_2025_10_06.csv', index=False)

## New Rule

### userRateGroupKeyByGradingRules

In [32]:
result_rbp_new = helpers.get_distribution_batch('user_rbp_2025_10_06.csv', 
                                           tok,
                                            dataset='transaction',
                                            orderid='reference',
                                           version=73,
                                           name='userRateGroupKeyByGradingRules',
                                            namespace='indodana.rules.general.implementation',
                                           is_staging=False,
                                           verbose=False,
#                                            v4=True,
                                           v5=True,
                                           max_file_size=45
                                          )

Checking function correctness... [OK]
45936 37.80470561981201
	>>> maximum batch size: 1500

GOOGLE_APPLICATION_CREDENTIALS: ../automatic-pancake/data-jupyter-notebook@athena-179008.json
GCS_BUCKET: cermati-indodana-risk-data-analytic
Connection initialized to Google Cloud Storage project athena-179008 and bucket cermati-indodana-risk-data-analytic

List of inputs:
1760331233_93bc1d13ea154dbe8312a63cb4aa5976_0.json: 42.00240230560303 MB
1760331233_93bc1d13ea154dbe8312a63cb4aa5976_1.json: 37.35114574432373 MB
1760331233_93bc1d13ea154dbe8312a63cb4aa5976_10.json: 38.22030258178711 MB
1760331233_93bc1d13ea154dbe8312a63cb4aa5976_11.json: 36.4429235458374 MB
1760331233_93bc1d13ea154dbe8312a63cb4aa5976_12.json: 38.793124198913574 MB
1760331233_93bc1d13ea154dbe8312a63cb4aa5976_13.json: 39.40442085266113 MB
1760331233_93bc1d13ea154dbe8312a63cb4aa5976_14.json: 36.752838134765625 MB
1760331233_93bc1d13ea154dbe8312a63cb4aa5976_15.json: 37.3422908782959 MB
1760331233_93bc1d13ea154dbe8312a63cb4aa597

  0%|          | 0/31 [00:00<?, ?it/s]

In [33]:
ids_rbp_new = result_rbp_new[0]
ids_rbp_new

['d672d96e-9d6f-4f50-855b-237415f7d67a',
 '8ab216e8-b029-422b-b19d-daa890e88e3b',
 '9c9e6334-98fe-47c7-8c4b-512fb4e73072',
 '40521f09-16d7-4269-9ea5-150fa1754270',
 'a910fdcd-a7d5-4180-b39e-4a69489dfbaa',
 '76e56d31-bc44-44c9-915b-c1f0fc976b7e',
 'e0da1aac-5f85-4adf-9821-88c1bb5a9a4a',
 'abbfce9c-f1de-4650-a17c-b1a3a13b9a92',
 '1958f838-d3b6-4a89-a3cd-da1a8a9ce862',
 'fc7adc9e-c16e-4a00-9ba6-df96130d777d',
 '4cb462a7-736f-4825-b1a9-f77507a344bf',
 '387548eb-d012-4502-a7e5-97f97ce46656',
 '9064bd18-e279-4958-99d9-da709f947693',
 '9467a70e-ca87-433f-9f84-78eab6711037',
 'c84d3bb2-91fa-48c1-b463-d8720fb90816',
 'e778907e-039a-476a-b0d3-b30b3c22f91d',
 'e5b93a8d-284c-4bf9-9e4a-83569a27e27d',
 '5607793a-ff4c-485d-ba54-bb27734631e7',
 '05c29150-0e1f-450d-ae58-e4f9706e32cd',
 '8e58abbb-4020-43aa-bf16-3e14fe445c26',
 'fac8926b-ac70-44bd-8007-35337165f0e2',
 '6ae0a80f-86ca-4105-a621-36d3de2762c2',
 'b9bb2172-fdef-4c07-8e6a-32f340c72f4b',
 '426a8223-4195-4cd7-bf95-7806d2e33f63',
 '2bc93ed2-0458-

In [34]:
success_rbp_new, output_rbp_new, input_rbp_new = helpers.get_evaluation_results_gcs(ids_rbp_new, 
                                                                        tok,
                                                                        path='user_rbp_2025_10_06.csv',
                                                                        v5=True
                                                                       )

GOOGLE_APPLICATION_CREDENTIALS: ../automatic-pancake/data-jupyter-notebook@athena-179008.json
GCS_BUCKET: cermati-indodana-risk-data-analytic
Connection initialized to Google Cloud Storage project athena-179008 and bucket cermati-indodana-risk-data-analytic


1. d672d96e-9d6f-4f50-855b-237415f7d67a
	>>> Evaluation status: DONE_PROCESSING_INPUT
	>>> Evaluation status: DONE
time:2025-10-13 04:55:43.504504
2. 8ab216e8-b029-422b-b19d-daa890e88e3b
	>>> Evaluation status: DONE
time:2025-10-13 04:55:46.047773
3. 9c9e6334-98fe-47c7-8c4b-512fb4e73072
	>>> Evaluation status: DONE
time:2025-10-13 04:55:49.032326
4. 40521f09-16d7-4269-9ea5-150fa1754270
	>>> Evaluation status: WAITING
	>>> Evaluation status: DONE
time:2025-10-13 04:56:51.923400
5. a910fdcd-a7d5-4180-b39e-4a69489dfbaa
{"success":false,"error":{"type":"GenericValidationError","message":"Bulk evaluate job with ID A910FDCD-A7D5-4180-B39E-4A69489DFBAA is not found","status":422}}
	>>> Evaluation status: False
{"success":false,"error":{"

In [35]:
results_df_rbp_new = pd.DataFrame(output_rbp_new)
results_df_rbp_new

,result,identifier,resultType
0,{'offline_merchant_risk_category_group': 'high...,WNB0YCX7,OBJECT
1,"{'offline_merchant_risk_category_group': 'ok',...",2K8LRWCD,OBJECT
2,{'offline_merchant_risk_category_group': 'high...,87FV6RZR,OBJECT
3,"{'offline_merchant_risk_category_group': 'ok',...",HEAJFM71,OBJECT
4,"{'offline_merchant_risk_category_group': 'ok',...",8XEQE4K4,OBJECT
...,...,...,...
45885,"{'offline_merchant_risk_category_group': 'ok',...",N2JGTEHJ,OBJECT
45886,"{'offline_merchant_risk_category_group': 'ok',...",KRAQCFXM,OBJECT
45887,"{'offline_merchant_risk_category_group': 'ok',...",QE8RXUUH,OBJECT
45888,"{'offline_merchant_risk_category_group': 'ok',...",F0GDFKMY,OBJECT


In [36]:
results_df_rbp_new.to_csv('results_user_rbp_v73_trx_2025_10_06.csv', index=False)

##  final df

In [57]:
for col in results_rbp_old_df.columns:
    if col not in ['identifier', 'orderId']:
        results_rbp_old_df.rename(columns={col: col + '_old'}, inplace=True)

In [58]:
for col in results_rbp_new_df.columns:
    if col not in ['identifier', 'orderId']:
        results_rbp_new_df.rename(columns={col: col + '_new'}, inplace=True)

In [40]:
# old data
final_df = results_rbp_old_df
# final_df = results_autoapprove_old_df

# new data
final_df = final_df.merge(results_rbp_new_df, how='left', on='identifier')

# original data
final_df = final_df.merge(data, how='left', left_on='identifier', right_on='reference', suffixes=('', '_ori_data'))

In [41]:
final_df['reference'].value_counts()

PLT-CBXHI7JP                     4
788085850007087-1758618105916    4
1322477820-1758173374332         4
FKO32510031932280101             4
370319417                        4
                                ..
IN-8UVXWGD                       1
IN-QCWIRLE                       1
IN-IM916ZJ                       1
IN-U50P1V9                       1
ACV9Y4J4                         1
Name: reference, Length: 91024, dtype: int64

In [42]:
final_df = final_df.drop_duplicates(subset=['reference'])

In [43]:
final_df['reference'].value_counts()

M2BMZ7Q5                                     1
plpr_040c712c-5309-4283-8251-2896397f696b    1
371007729                                    1
370641079                                    1
2510050439921138                             1
                                            ..
RSGPLTTY                                     1
66582231                                     1
MRV13QUZ                                     1
Q6YWGVT5                                     1
ACV9Y4J4                                     1
Name: reference, Length: 91024, dtype: int64

In [44]:
final_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91024 entries, 0 to 91372
Data columns (total 51 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   result_old                              91024 non-null  object 
 1   identifier                              91024 non-null  object 
 2   resultType_old                          91024 non-null  object 
 3   result_new                              45779 non-null  object 
 4   resultType_new                          45779 non-null  object 
 5   result_old_ori_data                     91024 non-null  object 
 6   identifier_ori_data                     91024 non-null  object 
 7   resultType_old_ori_data                 91024 non-null  object 
 8   result_new_ori_data                     45779 non-null  object 
 9   resultType_new_ori_data                 45779 non-null  object 
 10  id                                      91024 non-null  ob

In [45]:
final_df['result_old'].iloc[0]

"{'offline_merchant_risk_category_group': 'high risk', 'user_promo_risk_grade': 'C', 'rate_group_key': 'default', 'offline_item_category': 'smartphone', 'mandatory_dp_rate': 0, 'offline_merchant_tag': 'normal', 'offline_merchant_risk_category': 'not top merchant', 'is_offline_transaction': True, 'is_trx_allowed_to_use_promo': True, 'is_existing_user': False}"

In [46]:
final_df['result_old'].str.replace("'", '"').apply(ast.literal_eval)

0        {'offline_merchant_risk_category_group': 'high...
1        {'offline_merchant_risk_category_group': 'high...
2        {'offline_merchant_risk_category_group': 'high...
3        {'offline_merchant_risk_category_group': 'high...
4        {'offline_merchant_risk_category_group': 'high...
                               ...                        
91368    {'offline_merchant_risk_category_group': 'ok',...
91369    {'offline_merchant_risk_category_group': 'ok',...
91370    {'offline_merchant_risk_category_group': 'ok',...
91371    {'offline_merchant_risk_category_group': 'ok',...
91372    {'offline_merchant_risk_category_group': 'ok',...
Name: result_old, Length: 91024, dtype: object

In [47]:
final_df['result_old'].str.replace("'", '"').fillna('{}')

0        {"offline_merchant_risk_category_group": "high...
1        {"offline_merchant_risk_category_group": "high...
2        {"offline_merchant_risk_category_group": "high...
3        {"offline_merchant_risk_category_group": "high...
4        {"offline_merchant_risk_category_group": "high...
                               ...                        
91368    {"offline_merchant_risk_category_group": "ok",...
91369    {"offline_merchant_risk_category_group": "ok",...
91370    {"offline_merchant_risk_category_group": "ok",...
91371    {"offline_merchant_risk_category_group": "ok",...
91372    {"offline_merchant_risk_category_group": "ok",...
Name: result_old, Length: 91024, dtype: object

In [48]:
final_df['result_new']

0                                                      NaN
1        {'offline_merchant_risk_category_group': 'high...
2        {'offline_merchant_risk_category_group': 'high...
3        {'offline_merchant_risk_category_group': 'high...
4        {'offline_merchant_risk_category_group': 'high...
                               ...                        
91368    {'offline_merchant_risk_category_group': 'ok',...
91369    {'offline_merchant_risk_category_group': 'ok',...
91370    {'offline_merchant_risk_category_group': 'ok',...
91371    {'offline_merchant_risk_category_group': 'ok',...
91372    {'offline_merchant_risk_category_group': 'ok',...
Name: result_new, Length: 91024, dtype: object

In [49]:
def safe_literal_eval(node_string):
    """
    Safely evaluates a string that should be a Python literal (like a dict, list, etc.).
    If the string is malformed or another error occurs, it returns a default value,
    in this case, an empty dictionary {}.
    """
    # Ensure the input is a string before attempting to evaluate
    if isinstance(node_string, str):
        try:
            # The core of the logic: try to evaluate the string
            return ast.literal_eval(node_string)
        except (ValueError, SyntaxError):
            # If literal_eval fails, it raises ValueError or SyntaxError
            # We catch these and return a safe default.
            print(f"Error parsing string: '{node_string}'. Returning empty dict.")
            return {}
    # If it's not a string (e.g., already a dict, or NaN), return it as is.
    return node_string

In [53]:
final_df['rbp_result_dict_ori'] = final_df['result'].fillna('{}').apply(json.loads)
final_df['rate_group_key_ori'] = final_df['rbp_result_dict_ori'].str['rate_group_key']

final_df['rbp_result_dict_old'] = final_df['result_old'].str.replace("'", '"').fillna('{}').apply(ast.literal_eval)
final_df['rate_group_key_old'] = final_df['rbp_result_dict_old'].str['rate_group_key']

final_df['rbp_result_dict_new'] = final_df['result_new'].fillna('{}').str.replace("'", '"').apply(lambda x: safe_literal_eval(x))
final_df['rate_group_key_new'] = final_df['rbp_result_dict_new'].str['rate_group_key']
final_df['ab_test_va_rbp_new'] = final_df['rbp_result_dict_new'].str['ab_test_va_rbp']

In [67]:
final_df[final_df['identifier'] == '750881289904615-1759644745375']['rate_group_key_new'].iloc[0]

'1month-12month-p3-i3'

In [54]:
# final_df.to_csv('final_df_cli_indodana_increase_ar_mar_2024.csv', index=False)
final_df.to_csv('final_df_va_rbp_2025_10_13.csv', index=False)

In [52]:
final_df.shape

(91024, 51)

# Analysis prep

In [59]:
final_df_analysis.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91024 entries, 0 to 91023
Data columns (total 52 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   result_old                              91024 non-null  object 
 1   identifier                              91024 non-null  object 
 2   resultType_old                          91024 non-null  object 
 3   result_new                              45779 non-null  object 
 4   resultType_new                          45779 non-null  object 
 5   result_old_ori_data                     91024 non-null  object 
 6   identifier_ori_data                     91024 non-null  object 
 7   resultType_old_ori_data                 91024 non-null  object 
 8   result_new_ori_data                     45779 non-null  object 
 9   resultType_new_ori_data                 45779 non-null  object 
 10  id                                      91024 non-null  ob

# Simulation Results Analysis

In [60]:
final_df_analysis = final_df_analysis[final_df_analysis['userLastActivitySegment'] != '-9999']

In [61]:
final_df_analysis['information'] = final_df_analysis['information'].fillna('{}').apply(json.loads)

In [62]:
final_df_analysis['merchantId'] = final_df_analysis['information'].str['functionParameters'].str['merchantId']

In [77]:
final_df_analysis['sumPaymentVaBeforeTrx_ctg'] = '0. Other'
final_df_analysis.loc[final_df_analysis['sumPaymentVaBeforeTrx'].astype(float) > 0, 'sumPaymentVaBeforeTrx_ctg'] = '1. Have VA Payment'
final_df_analysis.loc[final_df_analysis['sumPaymentVaBeforeTrx'].astype(float) <= 0, 'sumPaymentVaBeforeTrx_ctg'] = '2. No VA Payment'

final_df_analysis['ab_test_va_rbp_new_ctg'] = 'Other'
final_df_analysis.loc[final_df_analysis['ab_test_va_rbp_new'].isna(), 'ab_test_va_rbp_new_ctg'] = 'Not in A/B'
final_df_analysis.loc[final_df_analysis['ab_test_va_rbp_new'].notna(), 'ab_test_va_rbp_new_ctg'] = final_df_analysis['ab_test_va_rbp_new']

In [64]:
VA_MERCHANTS = [
    '77337991-bf38-11ee-a258-31e5ebb41a87'
    ,'4ede4340-ab7b-11ee-9642-9f3f449c6c33'
    ,'826b91a3-a0b0-11ee-90cf-23cf2d34ee74'
    ,'184ab49f-ab7b-11ee-8196-87e351c1a74e'
    ,'209b4e8b-7324-11ee-9749-abb1dc7c3e41'
    ,'642cbedd-b99a-11ee-9b3e-fbaefd543359'
    ,'fe70c52c-7588-11ef-b98b-5754b239ed4d'
]

final_df_analysis['is_va_transaction'] = False
final_df_analysis.loc[final_df_analysis['merchantId'].isin(VA_MERCHANTS), 'is_va_transaction'] = True

In [65]:
cols = [
    'rate_group_key_old',
    'rate_group_key_new',
]

pivots=[]

for col in cols:
    index = [
        'is_va_transaction',
#         'ab_test_bq1_new',
#         'application_status',
        col
    ]
    piv_final = pd.pivot_table(final_df_analysis,
                               index=index,
                               values='reference',
                               aggfunc='count')
    piv_final['sum'] = piv_final['reference'].groupby(level=[0, 1]).transform('sum')
    piv_final['perc'] = piv_final['reference']/piv_final['sum']
    
    piv_final = pd.concat([piv_final], axis=1)
#     piv_final = piv_final.groupby(level=[2]).ffill()
    piv_final[['perc']] = piv_final[['perc']].apply(lambda x: x.map('{:,.2%}'.format))
    
    pivots.append(piv_final[[
        'reference',
        'sum', 
        'perc', 
    ]])

pivot_join = pd.concat(pivots, axis=1, keys=cols)

display(pivot_join)
# qgrid.show_grid(pivot_join, show_toolbar=True)

rate_group_key_old  \
                                                                         reference   
is_va_transaction                                                                    
False             1month-12month-no_min_installment-int199                    2235   
                  1month-12month-p3-i3                                       12334   
                  barcode_admin2%                                               10   
                  barcode_default                                              891   
                  combo_transaction_1                                           95   
                  default                                                     4532   
                  electric_vehicle                                               8   
                  klikindomaret_2%                                            1041   
                  offline_transaction190k                                     3193   
                  offline_transaction290k                                       95   
                  offline_transaction_additional_mp_199k_ir_399                  7   
                  offline_transaction_additional_mp_349k_ir_449                 59   
                  stellar-ewallet                                             5895   
                  tiket-paylater                                             15162   
True              1month-12month-p3-i3                                         168   
                  va_new_rate_2024_09                                            4   

                                                                        \
                                                                   sum   
is_va_transaction                                                        
False             1month-12month-no_min_installment-int199        2235   
                  1month-12month-p3-i3                           12334   
                  barcode_admin2%                                   10   
                  barcode_default                                  891   
                  combo_transaction_1                               95   
                  default                                         4532   
                  electric_vehicle                                   8   
                  klikindomaret_2%                                1041   
                  offline_transaction190k                         3193   
                  offline_transaction290k                           95   
                  offline_transaction_additional_mp_199k_ir_399      7   
                  offline_transaction_additional_mp_349k_ir_449     59   
                  stellar-ewallet                                 5895   
                  tiket-paylater                                 15162   
True              1month-12month-p3-i3                             168   
                  va_new_rate_2024_09                                4   

                                                                          \
                                                                    perc   
is_va_transaction                                                          
False             1month-12month-no_min_installment-int199       100.00%   
                  1month-12month-p3-i3                           100.00%   
                  barcode_admin2%                                100.00%   
                  barcode_default                                100.00%   
                  combo_transaction_1                            100.00%   
                  default                                        100.00%   
                  electric_vehicle                               100.00%   
                  klikindomaret_2%                               100.00%   
                  offline_transaction190k                        100.00%   
                  offline_transaction290k                        100.00%   
                  offline_transact

In [91]:
10/164 * 100

6.097560975609756

In [72]:
cols = [
    'rate_group_key_old',
    'rate_group_key_new',
]

pivots=[]

for col in cols:
    index = [
        'is_va_transaction',
        'sumPaymentVaBeforeTrx_ctg',
#         'ab_test_va_rbp_new',
#         'ab_test_bq1_new',
#         'application_status',
        col
    ]
    piv_final = pd.pivot_table(final_df_analysis[final_df_analysis['is_va_transaction'] == True],
                               index=index,
                               values='reference',
                               aggfunc='count')
    piv_final['sum'] = piv_final['reference'].groupby(level=[0, 1, 2]).transform('sum')
    piv_final['perc'] = piv_final['reference']/piv_final['sum']
    
    piv_final = pd.concat([piv_final], axis=1)
#     piv_final = piv_final.groupby(level=[2]).ffill()
    piv_final[['perc']] = piv_final[['perc']].apply(lambda x: x.map('{:,.2%}'.format))
    
    pivots.append(piv_final[[
        'reference',
        'sum', 
        'perc', 
    ]])

pivot_join = pd.concat(pivots, axis=1, keys=cols)

display(pivot_join)
# qgrid.show_grid(pivot_join, show_toolbar=True)

rate_group_key_old  \
                                                                          reference   
is_va_transaction sumPaymentVaBeforeTrx_ctg                                           
True              1. Have VA Payment        1month-12month-p3-i3                131   
                                            va_new_rate_2024_09                   2   
                  2. No VA Payment          1month-12month-p3-i3                 37   
                                            va_new_rate_2024_09                   2   

                                                                       \
                                                                  sum   
is_va_transaction sumPaymentVaBeforeTrx_ctg                             
True              1. Have VA Payment        1month-12month-p3-i3  131   
                                            va_new_rate_2024_09     2   
                  2. No VA Payment          1month-12month-p3-i3   37   
                                            va_new_rate_2024_09     2   

                                                                           \
                                                                     perc   
is_va_transaction sumPaymentVaBeforeTrx_ctg                                 
True              1. Have VA Payment        1month-12month-p3-i3  100.00%   
                                            va_new_rate_2024_09   100.00%   
                  2. No VA Payment          1month-12month-p3-i3  100.00%   
                                            va_new_rate_2024_09   100.00%   

                                                                 rate_group_key_new  \
                                                                          reference   
is_va_transaction sumPaymentVaBeforeTrx_ctg                                           
True              1. Have VA Payment        1month-12month-p3-i3                131   
                                            va_new_rate_2024_09                   2   
                  2. No VA Payment          1month-12month-p3-i3                 27   
                                            va_new_rate_2024_09                  12   

                                                                                
                                                                  sum     perc  
is_va_transaction sumPaymentVaBeforeTrx_ctg                                     
True              1. Have VA Payment        1month-12month-p3-i3  131  100.00%  
                                            va_new_rate_2024_09     2  100.00%  
                  2. No VA Payment          1month-12month-p3-i3   27  100.00%  
                                            va_new_rate_2024_09    12  100.00%

In [73]:
12/39 * 100

30.76923076923077

In [74]:
10/35 * 100

28.57142857142857

In [80]:
cols = [
    'rate_group_key_old',
    'rate_group_key_new',
]

pivots=[]

for col in cols:
    index = [
        'is_va_transaction',
        'sumPaymentVaBeforeTrx_ctg',
        'ab_test_va_rbp_new_ctg',
#         'ab_test_bq1_new',
#         'application_status',
        col
    ]
    piv_final = pd.pivot_table(final_df_analysis[final_df_analysis['is_va_transaction'] == True],
                               index=index,
                               values='reference',
                               aggfunc='count')
    piv_final['sum'] = piv_final['reference'].groupby(level=[0, 1]).transform('sum')
    piv_final['perc'] = piv_final['reference']/piv_final['sum']
    
    piv_final = pd.concat([piv_final], axis=1)
#     piv_final = piv_final.groupby(level=[2]).ffill()
    piv_final[['perc']] = piv_final[['perc']].apply(lambda x: x.map('{:,.2%}'.format))
    
    pivots.append(piv_final[[
        'reference',
        'sum', 
        'perc', 
    ]])

pivot_join = pd.concat(pivots, axis=1, keys=cols)

display(pivot_join)
# qgrid.show_grid(pivot_join, show_toolbar=True)

rate_group_key_old  \
                                                                                                 reference   
is_va_transaction sumPaymentVaBeforeTrx_ctg ab_test_va_rbp_new_ctg                                           
True              1. Have VA Payment        Not in A/B             1month-12month-p3-i3              131.0   
                                                                   va_new_rate_2024_09                 2.0   
                  2. No VA Payment          HIGHER                 1month-12month-p3-i3               10.0   
                                                                   va_new_rate_2024_09                 NaN   
                                            NORMAL                 1month-12month-p3-i3                7.0   
                                            Not in A/B             1month-12month-p3-i3               20.0   
                                                                   va_new_rate_2024_09                 2.0   

                                                                                                \
                                                                                           sum   
is_va_transaction sumPaymentVaBeforeTrx_ctg ab_test_va_rbp_new_ctg                               
True              1. Have VA Payment        Not in A/B             1month-12month-p3-i3  133.0   
                                                                   va_new_rate_2024_09   133.0   
                  2. No VA Payment          HIGHER                 1month-12month-p3-i3   39.0   
                                                                   va_new_rate_2024_09     NaN   
                                            NORMAL                 1month-12month-p3-i3   39.0   
                                            Not in A/B             1month-12month-p3-i3   39.0   
                                                                   va_new_rate_2024_09    39.0   

                                                                                                 \
                                                                                           perc   
is_va_transaction sumPaymentVaBeforeTrx_ctg ab_test_va_rbp_new_ctg                                
True              1. Have VA Payment        Not in A/B             1month-12month-p3-i3  98.50%   
                                                                   va_new_rate_2024_09    1.50%   
                  2. No VA Payment          HIGHER                 1month-12month-p3-i3  25.64%   
                                                                   va_new_rate_2024_09      NaN   
                                            NORMAL                 1month-12month-p3-i3  17.95%   
                                            Not in A/B             1month-12month-p3-i3  51.28%   
                                                                   va_new_rate_2024_09    5.13%   

                                                                                        rate_group_key_new  \
                                                                                                 reference   
is_va_transaction sumPaymentVaBeforeTrx_ctg ab_test_va_rbp_new_ctg                                           
True              1. Have VA Payment        Not in A/B             1month-12month-p3-i3              131.0   
                                                                   va_new_rate_2024_09                 2.0   
                  2. No VA Payment          HIGHER                 1month-12month-p3-i3                NaN   
                                                                   va_new_rate_2024_09                10.0   
                                            NORMAL                 1month-12month-p3-i3                7.0   
                                            Not in A/B             1month-12month-p3-i3               20.0   
                         

In [75]:
10 / 17 * 100

58.82352941176471

In [83]:
cols = [
    'rate_group_key_old',
    'rate_group_key_new',
]

pivots=[]

for col in cols:
    index = [
        'is_va_transaction',
        'sumPaymentVaBeforeTrx_ctg',
        'userLastActivitySegment',
        'hasNonLimitActiveCashLoanContract',
        'ab_test_va_rbp_new_ctg',
#         'ab_test_bq1_new',
#         'application_status',
        col
    ]
    piv_final = pd.pivot_table(final_df_analysis[final_df_analysis['is_va_transaction'] == True],
                               index=index,
                               values='reference',
                               aggfunc='count')
    piv_final['sum'] = piv_final['reference'].groupby(level=[0, 1, 2]).transform('sum')
    piv_final['perc'] = piv_final['reference']/piv_final['sum']
    
    piv_final = pd.concat([piv_final], axis=1)
#     piv_final = piv_final.groupby(level=[2]).ffill()
    piv_final[['perc']] = piv_final[['perc']].apply(lambda x: x.map('{:,.2%}'.format))
    
    pivots.append(piv_final[[
        'reference',
        'sum', 
        'perc', 
    ]])

pivot_join = pd.concat(pivots, axis=1, keys=cols)

display(pivot_join)
# qgrid.show_grid(pivot_join, show_toolbar=True)

rate_group_key_old  \
                                                                                                                                                               reference   
is_va_transaction sumPaymentVaBeforeTrx_ctg userLastActivitySegment     hasNonLimitActiveCashLoanContract ab_test_va_rbp_new_ctg                                           
True              1. Have VA Payment        Blibli Paylater             False                             Not in A/B             1month-12month-p3-i3                4.0   
                                            CASH_LOAN                   False                             Not in A/B             1month-12month-p3-i3               12.0   
                                                                        True                              Not in A/B             1month-12month-p3-i3                6.0   
                                            Digital Products by Cermati False                             Not in A/B             1month-12month-p3-i3                5.0   
                                                                                                                                 va_new_rate_2024_09                 1.0   
                                                                        True                              Not in A/B             1month-12month-p3-i3                5.0   
                                            E-Wallet by Cermati         False                             Not in A/B             1month-12month-p3-i3                6.0   
                                                                        True                              Not in A/B             1month-12month-p3-i3                1.0   
                                            Offline                     False                             Not in A/B             1month-12month-p3-i3                2.0   
                                                                        True                              Not in A/B             1month-12month-p3-i3                1.0   
                                            Other Online Indodana       False                             Not in A/B             1month-12month-p3-i3                1.0   
                                                                        True                              Not in A/B             1month-12month-p3-i3                1.0   
                                            Tiket Paylater              False                             Not in A/B             1month-12month-p3-i3                1.0   
                                                                        True                              Not in A/B             1month-12month-p3-i3                2.0   
                                            VA                          False                             Not in A/B             1month-12month-p3-i3               54.0   
                                                                                                                                 va_new_rate_2024_09                 1.0   
                                                                        True                              Not in A/B             1month-12month-p3-i3               30.0   
                  2. No VA Payment          CASH_LOAN                   False                             HIGHER                 1month-12month-p3-i3                4.0   
                                                                                                                                 va_new_rate_2024_09                 NaN   
                                                                                                          NORMAL                 1month-12month-p3-i3                5.0   
                                                                                                          Not in A/B             va_new_rate_2024_09                 2.0   
                      

In [84]:
4/9*100

44.44444444444444

In [69]:
cols = [
    'rate_group_key_old',
    'rate_group_key_new',
]

pivots=[]

for col in cols:
    index = [
        'is_va_transaction',
        'sumPaymentVaBeforeTrx_ctg',
        'hasNonLimitActiveCashLoanContract',
#         'ab_test_bq1_new',
#         'application_status',
        col
    ]
    piv_final = pd.pivot_table(final_df_analysis[final_df_analysis['is_va_transaction'] == True],
                               index=index,
                               values='reference',
                               aggfunc='count')
    piv_final['sum'] = piv_final['reference'].groupby(level=[0, 1, 2]).transform('sum')
    piv_final['perc'] = piv_final['reference']/piv_final['sum']
    
    piv_final = pd.concat([piv_final], axis=1)
#     piv_final = piv_final.groupby(level=[2]).ffill()
    piv_final[['perc']] = piv_final[['perc']].apply(lambda x: x.map('{:,.2%}'.format))
    
    pivots.append(piv_final[[
        'reference',
        'sum', 
        'perc', 
    ]])

pivot_join = pd.concat(pivots, axis=1, keys=cols)

display(pivot_join)
# qgrid.show_grid(pivot_join, show_toolbar=True)

rate_group_key_old  \
                                                                                                            reference   
is_va_transaction sumPaymentVaBeforeTrx_ctg hasNonLimitActiveCashLoanContract                                           
True              1. Have VA Payment        False                             1month-12month-p3-i3               85.0   
                                                                              va_new_rate_2024_09                 2.0   
                                            True                              1month-12month-p3-i3               46.0   
                  2. No VA Payment          False                             1month-12month-p3-i3               29.0   
                                                                              va_new_rate_2024_09                 2.0   
                                            True                              1month-12month-p3-i3                8.0   
                                                                              va_new_rate_2024_09                 NaN   

                                                                                                          \
                                                                                                     sum   
is_va_transaction sumPaymentVaBeforeTrx_ctg hasNonLimitActiveCashLoanContract                              
True              1. Have VA Payment        False                             1month-12month-p3-i3  87.0   
                                                                              va_new_rate_2024_09   87.0   
                                            True                              1month-12month-p3-i3  46.0   
                  2. No VA Payment          False                             1month-12month-p3-i3  31.0   
                                                                              va_new_rate_2024_09   31.0   
                                            True                              1month-12month-p3-i3   8.0   
                                                                              va_new_rate_2024_09    NaN   

                                                                                                             \
                                                                                                       perc   
is_va_transaction sumPaymentVaBeforeTrx_ctg hasNonLimitActiveCashLoanContract                                 
True              1. Have VA Payment        False                             1month-12month-p3-i3   97.70%   
                                                                              va_new_rate_2024_09     2.30%   
                                            True                              1month-12month-p3-i3  100.00%   
                  2. No VA Payment          False                             1month-12month-p3-i3   93.55%   
                                                                              va_new_rate_2024_09     6.45%   
                                            True                              1month-12month-p3-i3  100.00%   
                                                                              va_new_rate_2024_09       NaN   

                                                                                                   rate_group_key_new  \
                                                                                                            reference   
is_va_transaction sumPaymentVaBeforeTrx_ctg hasNonLimitActiveCashLoanContract                                           
True              1. Have VA Payment        False                             1month-12month-p3-i3                 85   
                                                                              va_new_rate_2024_09                   2   
                                            True                              1month-12mont

# Sampling

In [57]:
final_df_analysis[
    (final_df_analysis['is_va_transaction'] == True)
    & (final_df_analysis['sumPaymentVaBeforeTrx'] <= 0)
    & (final_df_analysis['userLastActivitySegment'] == 'CASH_LOAN')
    & (final_df_analysis['rate_group_key_new'] != 'va_new_rate_2024_09')
][[
    'reference',
    'merchant_name',
    'userLastActivitySegment',
    'rate_group_key_ori',
    'rate_group_key_old',
    'rate_group_key_new',
    'sumPaymentVaBeforeTrx'

]].reset_index(drop=True)

,reference,merchant_name,userLastActivitySegment,rate_group_key_ori,rate_group_key_old,rate_group_key_new,sumPaymentVaBeforeTrx
0,750881289904615-1759644745375,Belanja di Tokopedia,CASH_LOAN,1month-12month-p3-i3,1month-12month-p3-i3,NaN,0.0
1,788082115810831-1759318704006,Belanja di Shopee,CASH_LOAN,1month-12month-p3-i3,1month-12month-p3-i3,NaN,0.0
2,788089508815259-1759335145767,Belanja di Shopee,CASH_LOAN,1month-12month-p3-i3,1month-12month-p3-i3,NaN,0.0
3,750800195572809-1759332020358,Belanja di Tokopedia,CASH_LOAN,1month-12month-p3-i3,1month-12month-p3-i3,NaN,0.0
4,788087775977670-1759302739398,Belanja di Shopee,CASH_LOAN,1month-12month-p3-i3,1month-12month-p3-i3,NaN,0.0
5,788085779396168-1759340272052,Belanja di Shopee,CASH_LOAN,1month-12month-p3-i3,1month-12month-p3-i3,NaN,0.0
6,788082115810831-1759318568033,Belanja di Shopee,CASH_LOAN,1month-12month-p3-i3,1month-12month-p3-i3,NaN,0.0
7,788081313903381-1758787399913,Belanja di Shopee,CASH_LOAN,1month-12month-p3-i3,1month-12month-p3-i3,NaN,0.0
8,750800119064273-1759653363768,Belanja di Tokopedia,CASH_LOAN,1month-12month-p3-i3,1month-12month-p3-i3,NaN,0.0
9,788089508815259-1759342230998,Belanja di Shopee,CASH_LOAN,1month-12month-p3-i3,1month-12month-p3-i3,NaN,0.0


In [330]:
final_df_analysis[
    (final_df_analysis['ab_test_bq1_new'] == 'indodana_normal_set')
][[
    'orderId',
    'business_segment',
    'submitDate',
    'cash_origination',
    'ab_test_bq1',
    'ab_test_bq1_new',
    'application_status',
    'product_type',
    'qualified_new'
]]['application_status'].value_counts(dropna=False)

Not Qualified    208
Name: application_status, dtype: int64

In [351]:
data[data['orderId'] == 'ATH-T3Z7T64C'][[
    'orderId',
    'pefindo_score',
    'new_pefindo_score_cash_existing'
]]

,orderId,pefindo_score,new_pefindo_score_cash_existing
10833,ATH-T3Z7T64C,408.0,408.0


In [364]:
data[(data['pefindo_score'].notna()) & (data['risk_analytics_pefindo_score'] > data['pefindo_score'])][[
    'orderId',
    'cash_origination',
    'ab_test_bq1',
    'pefindo_score',
    'risk_analytics_pefindo_score',
    'application_status'
]]['application_status'].value_counts(dropna=False)

Approved         747
Not Qualified    562
Rejected           3
Name: application_status, dtype: int64

In [365]:
data[(data['pefindo_score'].notna()) & (data['risk_analytics_pefindo_score'] < data['pefindo_score'])][[
    'orderId',
    'cash_origination',
    'ab_test_bq1',
    'pefindo_score',
    'risk_analytics_pefindo_score',
    'application_status'
]]['application_status'].value_counts(dropna=False)

Approved         3310
Not Qualified    3055
Rejected           16
Cancelled           1
Incomplete          1
Name: application_status, dtype: int64

In [148]:
final_df_analysis[final_df_analysis['orderId'].isin(['ATH-A22WGXFR', 'ATH-RNN54MNA', 'ATH-CXLVNG8X', 'ATH-H29AP43E', 'ATH-A8Z19JB9'])][[
    'orderId',
    'approval_tag_new',
    'approved_amount_new',
    'tenure_new',
    'max_amount_new',
    'application_status',
    'contract_status',
]]

,orderId,approval_tag_new,approved_amount_new,tenure_new,max_amount_new,application_status,contract_status
69,ATH-A22WGXFR,H4B,2000000.0,3,2000000.0,Not Qualified,NaN
81,ATH-RNN54MNA,H1,3000000.0,3,3000000.0,Not Qualified,NaN
125,ATH-CXLVNG8X,H4B,4000000.0,6,4000000.0,Not Qualified,NaN
134,ATH-H29AP43E,H4B,4000000.0,6,4000000.0,Not Qualified,NaN
150,ATH-A8Z19JB9,H4B,2000000.0,3,2000000.0,Not Qualified,NaN


In [150]:
final_df_analysis[
    (final_df_analysis['has_approval_tag_ori'] == True)
    & (final_df_analysis['has_approval_tag_new'] == True)
][[
    'orderId',
    'contract_status',
    'max_amount_ori'
]]

,orderId,contract_status,max_amount_ori
1,ATH-639ZA29Y,WAITING_USER_APPROVAL,3000000.0
2,ATH-PPD817UK,WAITING_USER_APPROVAL,25000000.0
4,ATH-7WKEL5M9,WAITING_USER_APPROVAL,25000000.0
5,ATH-NEDGMEZX,WAITING_USER_APPROVAL,5000000.0
6,ATH-H8PURN1T,WAITING_USER_APPROVAL,15000000.0
...,...,...,...
17831,ATH-B2NK4VGK,WAITING_USER_APPROVAL,8000000.0
17832,ATH-AZC316ZQ,WAITING_USER_APPROVAL,3000000.0
17833,ATH-7KXXQSEN,WAITING_USER_APPROVAL,3000000.0
17835,ATH-MT4UN1QP,WAITING_USER_APPROVAL,3000000.0


In [154]:
final_df_analysis[final_df_analysis['orderId'] == 'ATH-U5LY81QG'][[
    'orderId',
    'application_status',
    'pefindo_score',
    'risk_analytics_pefindo_score',
    'is_qualified_new',
    'result_bq1_new',
    'result_bq2_new',
    'result_aa_new'
]]

,orderId,application_status,pefindo_score,risk_analytics_pefindo_score,is_qualified_new,result_bq1_new,result_bq2_new,result_aa_new
7068,ATH-U5LY81QG,Not Qualified,370.0,352.0,No,;;;;;;;;,JA01;;;;;;;,H1;3000000;3000000;3;;;;1;average;;


# Appendix

## Fetch Sample Data

In [15]:
%%bigquery data --project athena-179008 

WITH 
pt_ir_data AS (
  SELECT
    pt.id AS pt_id,
    pt.merchant_order_id,
    pt.created_at AS pt_created_at,
    FORMAT_TIMESTAMP('%Y/%m/%d', pt.created_at) AS pt_created_date,
    FORMAT_TIMESTAMP('%Y-%m', pt.created_at) AS pt_created_month,

    pt.amount AS pt_amount,
    pt.status AS pt_status,
    pt.fraud_decision AS pt_fraud_decision,
    pt.transaction_channel AS pt_transaction_channel,

    merc.name AS merchant_name,
    CASE 
      WHEN pt.transaction_channel = 'WHITELABEL' THEN merc.name
      WHEN pt.transaction_channel = 'OFFLINE' THEN 'Offline'
      WHEN pt.transaction_channel = 'QR_POS' THEN 'Offline'
      WHEN lower(merc.name) like '%alfamart%' or lower(merc.name) like '%alfamidi%' or lower(merc.name) like '%youtap%' or lower(merc.name) like '%fokus solusi proteksi%' or lower(merc.name) like '%bank sahabat sampoerna%' THEN 'Offline' 
      WHEN merc.name IN ('Tokopedia', 'BUKALAPAK Marketplace', 'Digital Products by Cermati') THEN merc.name
      ELSE 'Other online'
    END AS merchant_name_group,

    ir.id AS ir_id,
    ir.entity_id,
    ir.created_at AS ir_created_at,
    ir.status AS ir_status,
    ir.entity_type,
    
    cla.fee_scheme,
    cla.id AS cla_id,
    cla.status AS cla_status,
    cla.limit AS cla_limit,
    cla.created_at AS cla_created_at,

    (SAFE_DIVIDE((lc.principal + lc.interest), lc.loan_amount) - 1) / lc.tenure AS monthly_rate,
  FROM `data-platform-multifinance.vayu.tmf_merchant_purchase_transactions` pt
  LEFT JOIN `data-platform-multifinance.vayu.tmf_merchant_merchants` merc on merc.id = pt.merchant_id
  LEFT JOIN `data-platform-multifinance.vayu.tmf_merchant_purchase_transaction_installment_requests` ptir ON ptir.purchase_transaction_id = pt.id
  LEFT JOIN `data-platform-lending-infra.vayu.lendinginfra_credit_installment_requests` ir ON ir.entity_id = ptir.entity_id
  LEFT JOIN `data-platform-lending-infra.vayu.lendinginfra_credit_credit_limit_accounts` cla ON cla.id = ir.credit_limit_account_id
  LEFT JOIN `data-platform-lending-infra.vayu.lendinginfra_credit_loan_contracts` lc ON lc.installment_request_id = ir.id
),

rule_engine_result_user_rate_group_key AS ( 
  SELECT 
    *,
    row_number() OVER (PARTITION BY namespace, reference
                        ORDER BY abs_diff_run_log_to_trx_date_in_second ASC 
                      ) AS rank
   FROM
     ( SELECT 
        flogs.id,
        flogs.function_id,
        f.namespace,
        flogs.result,
        flogs.reference,
        flogs.version,
        flogs.information,
        flogs.created_at AS flogs_created_at,
    
        ABS(DATETIME_DIFF(flogs.created_at, pt_created_at, SECOND)) AS abs_diff_run_log_to_trx_date_in_second,

        TRIM(JSON_VALUE(information,'$.functionParameters.hasNonLimitActiveCashLoanContract'),'"') AS hasNonLimitActiveCashLoanContract,

      -- offlineFeatures userTransactionHistoryFeatures
        IFNULL(TRIM(JSON_VALUE(information,'$.functionParameters.offlineFeatures.userTransactionHistoryFeatures.userLastActivitySegment'),'"'), '-9999') AS userLastActivitySegment,
        IFNULL(CAST(TRIM(JSON_VALUE(information,'$.functionParameters.offlineFeatures.userTransactionHistoryFeatures.sumPaymentVaBeforeTrx'),'"') AS FLOAT64), -9999) AS sumPaymentVaBeforeTrx,

      FROM data-platform-lending-infra.vayu.lendinginfra_ruleengine_function_runtime_log_partitions flogs
      LEFT JOIN data-platform-lending-infra.vayu.lendinginfra_ruleengine_functions f ON flogs.function_id = f.id
      LEFT JOIN pt_ir_data ON pt_ir_data.merchant_order_id = flogs.reference
      WHERE f.name IN ('userRateGroupKeyByGradingRules') 
        AND (
          (flogs.created_at >= '2025-10-02') AND (flogs.created_at < '2025-10-06')
        )
        AND flogs.reference IS NOT NULL
     ) 
),

final_data AS (
  SELECT
    re_rbp.*,
    ptid.*,

  FROM rule_engine_result_user_rate_group_key re_rbp
  LEFT JOIN pt_ir_data ptid ON ptid.merchant_order_id = re_rbp.reference
  WHERE re_rbp.rank = 1
)

select 
  *
FROM final_data

Executing query with job ID: 9d249848-9db3-406d-b538-53479ee35ba2
Query executing: 12.09s

KeyboardInterrupt: 

In [23]:
data.to_csv('va_rbp_trx_2025_10_06.csv', index=False)

In [24]:
data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91139 entries, 0 to 91138
Data columns (total 35 columns):
 #   Column                                  Non-Null Count  Dtype              
---  ------                                  --------------  -----              
 0   id                                      91139 non-null  object             
 1   function_id                             91139 non-null  object             
 2   namespace                               91139 non-null  object             
 3   result                                  91139 non-null  object             
 4   reference                               91139 non-null  object             
 5   version                                 91139 non-null  Int64              
 6   information                             91139 non-null  object             
 7   flogs_created_at                        91139 non-null  datetime64[ns, UTC]
 8   abs_diff_run_log_to_trx_date_in_second  87843 non-null  Int64              
